In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv


In [2]:
import pandas as pd

# Define the common path to the files
data_path = '/kaggle/input/m5-forecasting-accuracy/'

# Load datasets using the common path
sales_train_validation = pd.read_csv(f'{data_path}sales_train_validation.csv')
sales_train_evaluation = pd.read_csv(f'{data_path}sales_train_evaluation.csv')
calendar = pd.read_csv(f'{data_path}calendar.csv')
sell_prices = pd.read_csv(f'{data_path}sell_prices.csv')
sample_submission = pd.read_csv(f'{data_path}sample_submission.csv')


In [3]:
# Melt the sales_train_validation data
sales_train_validation_melted = sales_train_validation.melt(
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='sales'
)

# Display the first few rows of the melted DataFrame
print("Melted Sales Train Validation Data:")
print(sales_train_validation_melted.head())

Melted Sales Train Validation Data:
                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id    d  sales  
0       CA  d_1      0  
1       CA  d_1      0  
2       CA  d_1      0  
3       CA  d_1      0  
4       CA  d_1      0  


In [4]:
def merge_in_chunks(df, calendar, prices, chunk_size=1500000):
    # List to hold the merged chunks
    merged_chunks = []
    
    # Number of chunks
    num_chunks = len(df) // chunk_size + 1
    
    for i in range(num_chunks):
        start = i * chunk_size
        end = (i + 1) * chunk_size
        chunk = df[start:end]
        chunk = chunk.merge(calendar, on='d', how='left')
        chunk = chunk.merge(prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
              
        merged_chunks.append(chunk)
        
        print(f"Chunk {i+1}/{num_chunks} merged")
    
    # Concatenate all chunks
    merged_df = pd.concat(merged_chunks, ignore_index=True)
    
    return merged_df

# Merge the melted DataFrame in chunks
sales_train_validation_merged = merge_in_chunks(sales_train_validation_melted, calendar, sell_prices)

# Display the first few rows of the merged DataFrame
print("Merged Sales Train Validation Data:")
print(sales_train_validation_merged.head())

Chunk 1/39 merged
Chunk 2/39 merged
Chunk 3/39 merged
Chunk 4/39 merged
Chunk 5/39 merged
Chunk 6/39 merged
Chunk 7/39 merged
Chunk 8/39 merged
Chunk 9/39 merged
Chunk 10/39 merged
Chunk 11/39 merged
Chunk 12/39 merged
Chunk 13/39 merged
Chunk 14/39 merged
Chunk 15/39 merged
Chunk 16/39 merged
Chunk 17/39 merged
Chunk 18/39 merged
Chunk 19/39 merged
Chunk 20/39 merged
Chunk 21/39 merged
Chunk 22/39 merged
Chunk 23/39 merged
Chunk 24/39 merged
Chunk 25/39 merged
Chunk 26/39 merged
Chunk 27/39 merged
Chunk 28/39 merged
Chunk 29/39 merged
Chunk 30/39 merged
Chunk 31/39 merged
Chunk 32/39 merged
Chunk 33/39 merged
Chunk 34/39 merged
Chunk 35/39 merged
Chunk 36/39 merged
Chunk 37/39 merged
Chunk 38/39 merged
Chunk 39/39 merged
Merged Sales Train Validation Data:
                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     

In [5]:
# Fill missing sell prices with the last available price
sales_train_validation_merged['sell_price'] = sales_train_validation_merged['sell_price'].ffill()


In [6]:
def create_rolling_features(df, window_sizes):
    for window in window_sizes:
        df[f'rolling_mean_{window}'] = df.groupby(['id'])['sales'].shift(1).rolling(window=window).mean()
        df[f'rolling_std_{window}'] = df.groupby(['id'])['sales'].shift(1).rolling(window=window).std()
    return df

# Apply rolling features
window_sizes = [7, 30]
sales_train_validation_merged = create_rolling_features(sales_train_validation_merged, window_sizes)

In [7]:
# Handle NaNs in rolling features
rolling_mean_cols = [f'rolling_mean_{window}' for window in window_sizes]
rolling_std_cols = [f'rolling_std_{window}' for window in window_sizes]

# Fill NaNs in rolling features with the overall mean and std (assuming that initial periods can use overall stats)
sales_train_validation_merged[rolling_mean_cols] = sales_train_validation_merged[rolling_mean_cols].fillna(sales_train_validation_merged['sales'].mean())
sales_train_validation_merged[rolling_std_cols] = sales_train_validation_merged[rolling_std_cols].fillna(sales_train_validation_merged['sales'].std())

Reduce Memory Usage by Downcasting
Here we convert data types to use less memory. For example, converting float64 to float32 and int64 to int32 can save memory.

In [8]:
def reduce_memory_usage(df):
    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

        else:
            df[col] = df[col].astype('category')

    return df

sales_train_validation_merged = reduce_memory_usage(sales_train_validation_merged)

In [9]:
#Check dataframe before reduction.
print(sales_train_validation_merged.columns)
print(sales_train_validation_merged.head())

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'rolling_mean_7',
       'rolling_std_7', 'rolling_mean_30', 'rolling_std_30'],
      dtype='object')
                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id    d  sales        date  wm_yr_wk  ... event_name_2  event_type_2  \
0       CA  d_1      0  2011-01-29     11101  ...          NaN    

Remove Unnecessary Columns
Drop any columns that aren't essential for the modeling process.

In [10]:
# List of columns to remove
columns_to_remove = [
    'dept_id', 'cat_id', 'state_id', 'weekday', 
    'wday', 'month', 'year', 'event_type_1', 
    'event_name_2', 'event_type_2'
]

# Remove the unwanted columns
sales_train_validation_merged = sales_train_validation_merged.drop(columns=columns_to_remove)

# Binary encoding: 1 if there's any event, 0 otherwise
sales_train_validation_merged['event_occurred'] = sales_train_validation_merged['event_name_1'].notna().astype(int)

# Drop the original 'event_name_1' column
sales_train_validation_merged = sales_train_validation_merged.drop('event_name_1', axis=1)

# Verify the final DataFrame
print(sales_train_validation_merged.head())
print(sales_train_validation_merged.columns)

                              id        item_id store_id    d  sales  \
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001     CA_1  d_1      0   
1  HOBBIES_1_002_CA_1_validation  HOBBIES_1_002     CA_1  d_1      0   
2  HOBBIES_1_003_CA_1_validation  HOBBIES_1_003     CA_1  d_1      0   
3  HOBBIES_1_004_CA_1_validation  HOBBIES_1_004     CA_1  d_1      0   
4  HOBBIES_1_005_CA_1_validation  HOBBIES_1_005     CA_1  d_1      0   

         date  wm_yr_wk  snap_CA  snap_TX  snap_WI  sell_price  \
0  2011-01-29     11101        0        0        0         NaN   
1  2011-01-29     11101        0        0        0         NaN   
2  2011-01-29     11101        0        0        0         NaN   
3  2011-01-29     11101        0        0        0         NaN   
4  2011-01-29     11101        0        0        0         NaN   

   rolling_mean_7  rolling_std_7  rolling_mean_30  rolling_std_30  \
0        1.125977       3.873047         1.125977        3.873047   
1        1.125977       3.873047

In [11]:
# Define the features and target
features = ['sell_price', 'wm_yr_wk', 'snap_CA', 'snap_TX', 'snap_WI', 
            'rolling_mean_7', 'rolling_mean_30', 'rolling_std_7', 'rolling_std_30']
target = 'sales'

In [12]:
models = {}  # Dictionary to store models and scalers for each department
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Get unique ids
id = sales_train_validation_merged['id'].unique()

# Calculate total number of items
total_items = len(id)

for idx, items in enumerate(id, start=1):
    # Filter data for the current department
    items_data = sales_train_validation_merged[sales_train_validation_merged['id'] == items]
    
    # Prepare features and target
    X = items_data[features]
    y = items_data[target]
    
    # Split data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Standardize the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    # Handle NaNs
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=features)
    X_train_scaled.fillna(X_train_scaled.mean(), inplace=True)
    X_val_scaled = pd.DataFrame(X_val_scaled, columns=features)
    X_val_scaled.fillna(X_val_scaled.mean(), inplace=True)
    X_train_scaled = X_train_scaled.values
    X_val_scaled = X_val_scaled.values
    
    # Initialize the model
    model = SGDRegressor(max_iter=1000, tol=1e-3)
    
    # Train the model on the entire dataset
    model.fit(X_train_scaled, y_train)
    
    # Store the model and scaler for the current department
    models[items] = (model, scaler)
    
    # Evaluate the model
    y_pred = model.predict(X_val_scaled)
    mse = mean_squared_error(y_val, y_pred)
    
    # Print progress every 5,000 items
    if idx % 5000 == 0 or idx == total_items:
        progress = (idx / total_items) * 100
        print(f"Progress: {progress:.2f}% ({idx}/{total_items} items processed)")

print("All items processed and models stored.")

Progress: 16.40% (5000/30490 items processed)
Progress: 32.80% (10000/30490 items processed)
Progress: 49.20% (15000/30490 items processed)
Progress: 65.60% (20000/30490 items processed)
Progress: 81.99% (25000/30490 items processed)
Progress: 98.39% (30000/30490 items processed)
Progress: 100.00% (30490/30490 items processed)
All items processed and models stored.


In [13]:
# Load the necessary files
calendar = pd.read_csv(f'{data_path}calendar.csv')
sell_prices = pd.read_csv(f'{data_path}sell_prices.csv')
submission_df = pd.read_csv(f'{data_path}sample_submission.csv')

# Convert 'date' column in calendar to datetime
calendar['date'] = pd.to_datetime(calendar['date'])

# Generate dates for the next 28 days
start_date = '2016-04-25'
next_28_days_dates = pd.date_range(start=start_date, periods=28)
next_28_days = pd.DataFrame({'date': next_28_days_dates})

# Merge with calendar data to get required features
next_28_days = next_28_days.merge(calendar, how='left', on='date')


In [14]:
# List of columns to remove
columns_to_remove = [
    'weekday', 'wday', 'month', 'year', 'event_type_1', 'event_name_2', 'event_type_2'
]
# Remove the unwanted columns
next_28_days = next_28_days.drop(columns=columns_to_remove)

# Binary encoding: 1 if there's any event, 0 otherwise
next_28_days['event_occurred'] = next_28_days['event_name_1'].notna().astype(int)

# Drop the original 'event_name_1' column
next_28_days = next_28_days.drop('event_name_1', axis=1)

unique_ids = sales_train_validation_merged['id'].unique()

next_28_days['id'] = unique_ids[:len(next_28_days)]

Bring back the sell_price to the 28 day data set because we reckon it as an important feature.

In [15]:
# Merge to add 'item_id' from sales_train_validation_merged to next_28_days
next_28_days = next_28_days.merge(sales_train_validation_merged[['id', 'item_id']], how='left', on='id')
next_28_days = next_28_days.merge(sell_prices[['item_id', 'wm_yr_wk', 'sell_price']], how='left', on=['item_id', 'wm_yr_wk'])

# Verify the merge
print(next_28_days.head())

        date  wm_yr_wk       d  snap_CA  snap_TX  snap_WI  event_occurred  \
0 2016-04-25     11613  d_1914        0        0        0               0   
1 2016-04-25     11613  d_1914        0        0        0               0   
2 2016-04-25     11613  d_1914        0        0        0               0   
3 2016-04-25     11613  d_1914        0        0        0               0   
4 2016-04-25     11613  d_1914        0        0        0               0   

                              id        item_id  sell_price  
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001        8.38  
1  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001        8.38  
2  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001        8.38  
3  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001        8.38  
4  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001        8.26  
